<a href="https://colab.research.google.com/github/KfirCohen-PyLab/Cloud-Computing-Elephant-team/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3'

import requests
import pandas as pd
from ipywidgets import widgets, VBox, Output
from IPython.display import display
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option("display.max_columns", None)

response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)
data_df = pd.DataFrame(data['result']['records'])

# Selection

In [30]:
output_area = Output()

# Create widgets
tozeret_nm_dropdown = widgets.Dropdown(
    options=[''] + sorted(data_df['tozeret_nm'].unique().tolist()),
    description='Tozeret:',
    style={'description_width': 'initial'}
)
kinuy_mishari_dropdown = widgets.Dropdown(
    options=[''],
    description='Kinuy Mishari:',
    style={'description_width': 'initial'}
)
# Function to update the second dropdown based on the selection of the first
def update_kinuy_mishari_options(change):
    if change['new']:  # Check if a valid option is selected
        filtered_values = data_df[data_df['tozeret_nm'] == change['new']]['kinuy_mishari'].unique()
        kinuy_mishari_dropdown.options = [''] + sorted(filtered_values)
    else:
        kinuy_mishari_dropdown.options = ['']

# Function to update the output area with the total records and unique 'ramat_gimur' values
def update_output(change=None):
    output_area.clear_output()
    selected_tozeret = tozeret_nm_dropdown.value
    selected_kinuy = kinuy_mishari_dropdown.value
    if selected_tozeret and selected_kinuy:
        filtered_df = data_df[
            (data_df['tozeret_nm'] == selected_tozeret) &
            (data_df['kinuy_mishari'] == selected_kinuy)
        ]
        total_records = len(filtered_df)
        unique_ramat_gimur = filtered_df['ramat_gimur'].unique()
        with output_area:
            print(f"Total Records: {total_records}")
            print(f"Unique Ramat Gimur: {', '.join(unique_ramat_gimur) if len(unique_ramat_gimur) > 0 else 'None'}")
    else:
        with output_area:
            print("Please select valid options for both dropdowns.")

# Observe changes in both dropdowns
tozeret_nm_dropdown.observe(update_kinuy_mishari_options, names='value')
kinuy_mishari_dropdown.observe(update_output, names='value')

display(VBox([tozeret_nm_dropdown, kinuy_mishari_dropdown, output_area]))

# Data

In [31]:
tab1_content = widgets.Output()
with tab1_content:
    print("Data Overview:")
    display(data_df.describe())  # Summary statistics

# Tab 2: Raw Data
tab2_content = widgets.Output()
with tab2_content:
    print("Raw Data:")
    display(data_df)  # Full DataFrame

# Tab 3: Charts
tab3_content = widgets.Output()
with tab3_content:
    year_freq = data_df['shnat_yitzur'].value_counts().sort_index()
    year_freq.plot(kind='bar')
    plt.title('Count of Shnat Yitzur')
    plt.xlabel('Shnat Yitzur')
    plt.ylabel('Frequency')
    plt.show()


tabs = widgets.Tab(children = [tab1_content, tab2_content, tab3_content])
tabs.set_title(0, 'Data Overview')
tabs.set_title(1, 'Raw Data')
tabs.set_title(2, 'Year Count')

# Display widgets and output area
display(tabs)